In [46]:
import numpy as np

### Activation Functions

In [47]:
# Base Activation Function Class
class Activation:
    def forward(self, z):
        """Forward pass for activation"""
        raise NotImplementedError

    def backward(self, z):
        """Backward pass for activation derivative"""
        raise NotImplementedError

# ReLU Activation
class ReLU(Activation):
    def forward(self, z):
        """ReLU activation: f(z) = max(0, z)"""
        return np.maximum(0, z)

    def backward(self, z):
        """Derivative of ReLU: 1 if z > 0, else 0"""
        return (z > 0).astype(float)

# Sigmoid Activation
class Sigmoid(Activation):
    def forward(self, z):
        """Sigmoid activation: f(z) = 1 / (1 + exp(-z))"""
        return 1 / (1 + np.exp(-z))

    def backward(self, z):
        """Derivative of Sigmoid"""
        s = self.forward(z) # Calculate sigmoid output during backward pass
        return s * (1 - s)

# Tanh Activation
class Tanh(Activation):
    def forward(self, z):
        """Tanh activation"""
        return np.tanh(z)

    def backward(self, z):
        """Derivative of Tanh"""
        return 1 - np.tanh(z) ** 2

# Softmax Activation
class Softmax(Activation):
    def forward(self, z):
        """Softmax activation for multi-class output"""
        exp_z = np.exp(z - np.max(z, axis=0, keepdims=True))  # Subtract max for numerical stability
        return exp_z / np.sum(exp_z, axis=0, keepdims=True)

    def backward(self, z):
        """Dummy derivative for softmax when used with crossentropy loss"""
        # The gradient for softmax is typically handled together with the cross-entropy loss
        # For backpropagation purposes with crossentropy, we return 1 here and the
        # combined gradient is calculated in the loss function's backward pass (or directly in the training loop).
        return np.ones_like(z)

# Linear Activation
class Linear(Activation):
    def forward(self, z):
        """Linear activation: f(z) = z"""
        return z

    def backward(self, z):
        """Derivative of linear activation: 1"""
        return np.ones_like(z)

# Leaky ReLU Activation
class LeakyReLU(Activation):
    def __init__(self, alpha=0.01):
        self.alpha = alpha

    def forward(self, z):
        """Leaky ReLU activation"""
        return np.where(z > 0, z, self.alpha * z)

    def backward(self, z):
        """Derivative of Leaky ReLU"""
        return np.where(z > 0, 1, self.alpha)

### Loss Functions

In [48]:
# Base Loss Function Class
class Loss:
    def compute_loss(self, y_pred, y_true):
        """Compute the loss"""
        raise NotImplementedError

    def backward(self, y_pred, y_true):
        """Compute the initial gradient for backpropagation"""
        raise NotImplementedError

# Mean Squared Error Loss
class MSE(Loss):
    def compute_loss(self, y_pred, y_true):
        """Mean Squared Error loss"""
        return np.mean((y_pred - y_true) ** 2) / 2

    def backward(self, y_pred, y_true):
        """Initial gradient for MSE"""
        return y_pred - y_true

# Categorical Crossentropy Loss
class CategoricalCrossentropy(Loss):
    def compute_loss(self, y_pred, y_true):
        """Categorical Crossentropy loss with clipping to avoid log(0)"""
        epsilon = 1e-15
        y_pred_clipped = np.clip(y_pred, epsilon, 1 - epsilon)
        return -np.mean(np.sum(y_true * np.log(y_pred_clipped), axis=0))

    def backward(self, y_pred, y_true):
        """Initial gradient for Categorical Crossentropy (often combined with Softmax)"""
        # For Softmax + Categorical Crossentropy, the combined gradient is simply y_pred - y_true
        return y_pred - y_true

### Optimizers

In [49]:
class Optimizer:
    def __init__(self, learning_rate):
        """Base optimizer class with learning rate"""
        self.learning_rate = learning_rate

    def update(self, layers, grads):
        pass

class GradientDescent(Optimizer):
    """Gradient Descent optimizer (mini-batch GD; SGD with batch_size=1)"""
    def update(self, layers, grads):
        """Update weights and biases using gradient descent"""
        for layer, (dW, db) in zip(layers, grads):
            layer.weights -= self.learning_rate * dW
            layer.biases -= self.learning_rate * db

class Adam(Optimizer):
    """Adam optimizer with momentum and RMSprop components"""
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        super().__init__(learning_rate)
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.v = {}  # First moment estimate
        self.s = {}  # Second moment estimate
        self.t = 0   # Time step

    def update(self, layers, grads):
        """Update weights and biases using Adam optimization"""
        self.t += 1
        for i, (layer, (dW, db)) in enumerate(zip(layers, grads)):
            if i not in self.v:
                self.v[i] = {'W': np.zeros_like(layer.weights), 'b': np.zeros_like(layer.biases)}
                self.s[i] = {'W': np.zeros_like(layer.weights), 'b': np.zeros_like(layer.biases)}
            # Update biased first moment estimate
            self.v[i]['W'] = self.beta1 * self.v[i]['W'] + (1 - self.beta1) * dW
            self.v[i]['b'] = self.beta1 * self.v[i]['b'] + (1 - self.beta1) * db
            # Update biased second raw moment estimate
            self.s[i]['W'] = self.beta2 * self.s[i]['W'] + (1 - self.beta2) * (dW ** 2)
            self.s[i]['b'] = self.beta2 * self.s[i]['b'] + (1 - self.beta2) * (db ** 2)
            # Compute bias-corrected estimates
            v_corr_W = self.v[i]['W'] / (1 - self.beta1 ** self.t)
            v_corr_b = self.v[i]['b'] / (1 - self.beta1 ** self.t)
            s_corr_W = self.s[i]['W'] / (1 - self.beta2 ** self.t)
            s_corr_b = self.s[i]['b'] / (1 - self.beta2 ** self.t)
            # Update parameters
            layer.weights -= self.learning_rate * v_corr_W / (np.sqrt(s_corr_W) + self.epsilon)
            layer.biases -= self.learning_rate * v_corr_b / (np.sqrt(s_corr_b) + self.epsilon)

### Layers

In [50]:
class DenseLayer:
    def __init__(self, input_size, output_size, activation):
        """Initialize a dense layer with weights, biases, and an activation object"""
        # Xavier/Glorot initialization
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(1.0 / input_size)
        self.biases = np.zeros((output_size, 1))
        self.activation = activation # Store an instance of an Activation class
        self.a_prev = None  # Previous layer's activation
        self.z = None       # Pre-activation value

    def forward(self, a_prev):
        """Forward propagation through the layer"""
        self.a_prev = a_prev
        self.z = self.weights @ a_prev + self.biases
        return self.activation.forward(self.z) # Use the activation object's forward method

    def backward(self, da):
        """Backward propagation to compute gradients"""
        batch_size = self.a_prev.shape[1]
        dz = da * self.activation.backward(self.z) # Use the activation object's backward method
        dW = (dz @ self.a_prev.T) / batch_size
        db = np.sum(dz, axis=1, keepdims=True) / batch_size
        da_prev = self.weights.T @ dz
        return da_prev, dW, db

### Utility Functions

In [51]:
class BatchGenerator:
    def __init__(self, X, y, batch_size):
        """Initialize BatchGenerator with data and batch size"""
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.num_samples = X.shape[1]
        self.indices = np.arange(self.num_samples)
        self.current_index = 0

    def __iter__(self):
        """Shuffle indices and reset current index for a new epoch"""
        np.random.shuffle(self.indices)
        self.current_index = 0
        return self

    def __next__(self):
        """Generate the next mini-batch"""
        if self.current_index >= self.num_samples:
            raise StopIteration

        end_index = min(self.current_index + self.batch_size, self.num_samples)
        batch_indices = self.indices[self.current_index:end_index]

        self.current_index = end_index

        return self.X[:, batch_indices], self.y[:, batch_indices]

### Neural Networks

In [52]:
class NeuralNetwork:
    def __init__(self, optimizer, loss_function, verbosity=1):
        """Initialize the network with an optimizer, loss function object, and verbosity level"""
        self.layers = []
        self.optimizer = optimizer
        self.loss_function = loss_function # Store an instance of a Loss class
        self.verbosity = verbosity # Add verbosity parameter

    def add_layer(self, layer):
        """Add a layer to the network"""
        self.layers.append(layer)

    def forward(self, X):
        """Forward propagation through all layers"""
        a = X
        for layer in self.layers:
            a = layer.forward(a)
        return a

    def compute_loss(self, y_pred, y_true):
        """Compute the loss using the loss function object"""
        return self.loss_function.compute_loss(y_pred, y_true)

    def train(self, X, y, epochs, batch_size):
        """Train the network using mini-batch processing"""
        n_batches = X.shape[1] // batch_size
        batch_generator = BatchGenerator(X, y, batch_size) # Create a BatchGenerator instance

        for epoch in range(epochs):
            # Process data in mini-batches
            batch_loss = 0
            for i, (X_batch, y_batch) in enumerate(batch_generator): # Iterate through the BatchGenerator
                y_pred = self.forward(X_batch)
                loss = self.compute_loss(y_pred, y_batch)
                batch_loss += loss

                # Print batch loss if verbosity is high enough
                if self.verbosity >= 2:
                    print(f"\nEpoch {epoch+1}, Batch {i+1}/{n_batches}")
                    print(f"  Loss: {loss:.6f}")
                    # Print y_pred for the first sample in the batch
                    if self.verbosity >= 2 and X_batch.shape[1] > 0:
                         print(f"  y_pred (first sample): {y_pred[:, 0]}")
                         print(f"  y_true (first sample): {y_batch[:, 0]}")


                # Initial gradient for output layer
                da = self.loss_function.backward(y_pred, y_batch) # Use loss object's backward method

                grads = []
                # Backpropagation through layers
                for layer in reversed(self.layers):
                    da, dW, db = layer.backward(da)
                    grads.append((dW, db))
                # Reverse the grads list to match the layer order
                grads.reverse()

                # Print summary of gradients if verbosity is high enough
                if self.verbosity >= 2:
                    print("  Gradients (per layer):")
                    for idx, (dW, db) in enumerate(grads):
                        print(f"    Layer {idx}: dW (mean={np.mean(dW):.6f}, std={np.std(dW):.6f}), "
                              f"db (mean={np.mean(db):.6f}, std={np.std(db):.6f})")


                # Update parameters
                self.optimizer.update(self.layers, grads)


            # Monitor progress (epoch loss)
            if self.verbosity >= 1 and (epoch % 10 == 0 or epoch == epochs - 1): # Print epoch loss every 10 epochs or on the last epoch
                y_pred_full = self.forward(X)
                loss_full = self.compute_loss(y_pred_full, y)
                print(f"\n=== Epoch {epoch+1} Summary ===")
                print(f"Average Batch Loss: {batch_loss / n_batches:.6f}")
                print(f"Full Dataset Loss: {loss_full:.6f}")
                if self.verbosity >= 2:
                    print(f"Final y_pred sample: {y_pred_full[:, 0]}")
                    print(f"Final y_true sample: {y[:, 0]}")

## Zoo Classification Training

In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from google.colab import drive


# drive.mount('/content/drive')

# --- Load and Preprocess Data ---
# Load zoo.csv from Google Drive
df = pd.read_csv('/content/drive/MyDrive/data_preprocessing_dataset/zoo.csv')

# Extract features and target
X = df.drop(['animal_name', 'class_type'], axis=1).values.T  # Shape: (16, n_samples)
y = df['class_type'].values.reshape(-1, 1)  # Shape: (n_samples, 1)

# One-hot encode the target (shift from 1-7 to 0-6)
encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(y).T  # Shape: (7, n_samples)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X.T, y_onehot.T, test_size=0.2, random_state=42)
X_train = X_train.T  # Shape: (16, n_train)
X_test = X_test.T    # Shape: (16, n_test)
y_train = y_train.T  # Shape: (7, n_train)
y_test = y_test.T    # Shape: (7, n_test)

# --- Create and Configure Neural Network ---
# Pass an instance of the optimizer and loss function classes
nn = NeuralNetwork(optimizer=Adam(learning_rate=0.01), loss_function=CategoricalCrossentropy(), verbosity=2)

# Add layers using instances of the activation classes
nn.add_layer(DenseLayer(16, 32, activation=ReLU()))
nn.add_layer(DenseLayer(32, 16, activation=ReLU()))
nn.add_layer(DenseLayer(16, 8, activation=ReLU()))
nn.add_layer(DenseLayer(8, 7, activation=Softmax()))

# --- Train the Network ---
print("Training the neural network...")
nn.train(X_train, y_train, epochs=200, batch_size=8)

# --- Evaluate on Test Set ---
y_pred = nn.forward(X_test)
accuracy = np.mean(np.argmax(y_pred, axis=0) == np.argmax(y_test, axis=0))
print(f"\nTest accuracy: {accuracy:.2f}")

Streaming output truncated to the last 5000 lines.
    Layer 2: dW (mean=-0.000001, std=0.000006), db (mean=-0.000000, std=0.000002)
    Layer 3: dW (mean=0.000000, std=0.000014), db (mean=0.000000, std=0.000002)

Epoch 155, Batch 10/10
  Loss: 0.000018
  y_pred (first sample): [9.99986084e-01 5.27700521e-23 1.39160328e-05 8.58260069e-16
 4.29585146e-23 1.83269651e-10 8.97238484e-26]
  y_true (first sample): [1. 0. 0. 0. 0. 0. 0.]
  Gradients (per layer):
    Layer 0: dW (mean=-0.000002, std=0.000007), db (mean=-0.000002, std=0.000004)
    Layer 1: dW (mean=-0.000001, std=0.000004), db (mean=-0.000001, std=0.000002)
    Layer 2: dW (mean=-0.000003, std=0.000012), db (mean=-0.000001, std=0.000002)
    Layer 3: dW (mean=-0.000000, std=0.000052), db (mean=-0.000000, std=0.000006)

Epoch 156, Batch 1/10
  Loss: 0.000032
  y_pred (first sample): [9.99999922e-01 8.15405241e-25 7.84501022e-08 6.46544183e-17
 1.03080995e-27 2.41122956e-12 1.88418423e-29]
  y_true (first sample): [1. 0. 0. 0. 0

In [55]:
# Get the features of the last animal in the original DataFrame
# Exclude 'animal_name' and 'class_type'
last_animal_features_df = df.tail(1).drop(['animal_name', 'class_type'], axis=1)

# Convert the DataFrame row to a NumPy array and transpose it
# to match the expected input shape (number_of_features, 1)
new_animal_features = last_animal_features_df.values.T # Shape (16, 1)


# Make a prediction for the new animal
prediction = nn.forward(new_animal_features)

# Interpret the prediction (get the class with the highest probability)
predicted_class_index = np.argmax(prediction, axis=0)

# The original class labels in the zoo dataset are 1-7.
# Our one-hot encoding shifted them to 0-6.
# So, we add 1 to the predicted index to get the original class number.
predicted_class = predicted_class_index[0] + 1

print(f"\nFeatures of the animal: {new_animal_features.T}")
print(f"The predicted class for the new animal is: {predicted_class}")

# You can compare this to the actual class of the last animal in the original df
actual_class = df.tail(1)['class_type'].values[0]
print(f"The actual class of the last animal is: {actual_class}")


Features of the animal: [[0 1 1 0 1 0 0 0 1 1 0 0 2 1 0 0]]
The predicted class for the new animal is: 2
The actual class of the last animal is: 2
